# 라이브러리를 안쓰는 프로젝트

In [ ]:
#얘네만 사용하기

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [ ]:
#데이터 불러오는 함수
'''
전역변수를 선언해서 거기에 데이터를 불러와서 저장한다.
변수명은 df이고 pd.read_csv를 사용함
'''

def load_data():
  global df
  urls = 'binary_dataset.csv'
  df = pd.read_csv(urls)

###파라미터와 편향 생성하기

In [ ]:


'''
가중치 초기화는 표준정규분포에서 뽑기
원래 기울기 초기화의 경우 역전파에서 가중치 소실이나 폭발이 일어나서 xavier나 he를 쓰지만 여기서는 역전파가 없으니 그냥 초기화
편향의 경우 0으로 초기화하거나 0.01로 초기화하는 경우가 많아서 0.01로 초기화를 해줌
전역변수인 weight와 bias에 할당하는데 가중치는 딥러닝에서 입력층의 개수이자 특성의 개수인 8개를 만들고 편향은 1개를 만들었음
'''

def para():
  global weight
  global bias
  weight = np.random.randn(8)
  bias     = 0.01


### 데이터 셔플해서 인덱스만 다시 설정하는 메서드

In [ ]:
'''
데이터를 분석에 사용하기 전에 한번 섞기
df.sample 함수를 사용한다.

sample 메서드는 DataFrame이나 Series에서 무작위로 몇개의 값(레이블)을 출력하는 메서드입니다.
df.sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)


n            : 추출할 갯수 입니다. replace가 False면 n의 최댓값은 레이블의 갯수를 넘을수 없습니다.
frac         : 추출할 비율입니다. 1보다 작은값으로 설정하며(예 : 0.3 이면 30%), n과 동시에 사용할 수 없습니다.
replace      : 중복추출의 허용 여부 입니다. True로 하면 중복추출이 가능하며 n의 최댓값이 레이블의 갯수보다 커도 됩니다.
weight       : 가중치입니다. 즉 레이블마다 추출될 확률을 지정할 수 있습니다. 합계가 1(100%)이 아닐경우 자동으로 1로 연산합니다.
random_state : 랜덤 추출한 값에 시드를 설정할 수 있습니다. 원하는 값을 설정하면, 항상 같은 결과를 출력합니다.
axis         : {0 : index / 1 : columns} 추출할 레이블입니다.
ignore_index : index의 무시 여부입니다. True일경우 출력시 index를 무시하고 숫자로 출력합니다.

이때 frac을 1로해서 전체를 뽑으니 무작위로 데이터를 전체로 뽑아서 셔플하는 기능을 가지고 있다. 
향후 편하기 사용하기 위해서 인덱스를 초기화한다.
기존 df를 섞은 df_shu를 전역변수로 선언해서 할당

'''


def data_shu():
  global df_shu
  df_shu = df.sample(frac=1).reset_index(drop = True)
  


### 데이터를 학습데이터와 테스트 데이터로 나누기

In [ ]:
#학습데이터와 테스트 데이터를 0.8 0.2 로 나누기
#20개중에서 무작위로 4개를 뽑고 얘는 테스트 데이터로 삼자


'''
먼저 빈 리스트 li를 선언하고 start와 end를 정한다. 이때 end는 데이터프레임의 길이이다. 
test_는 테스트 할 비율로 여기서는 0.2만큼쓰니 데이터 프레임의 크기에 0.2를 곱한 것만큼 테스틀 삼으니 test_는 4이다.
그럼 df_shu에서 랜덤하게 4개를 뽑아서 얘를 테스트 데이터로 삼음 된다. 먼저 반복문을 정하고 실행은 4번 한다.(테스트 4개를 뽑으니) 이후에 하나를 전체 df_shu에서 뽑는다.
똑같은 것이 이후에도 뽑히는 것을 방지하기 위해서 뽑고 리스트에 넣은 다음에 while반복문으로 li리스트에 있다면 다시 테스트 데이터를 뽑아서 리스트에 없다면 그걸 li에 추가한다.
그럼 li에는 테스트 데이터로 삼을 인덱스 번호가 저장된다. 이후에 df_shu에 그 인덱스만 따로 뽑아서 test로 삼고 테스트인덱스를 제외한 것이 train데이터가 된다.

'''


def data_split():
  li = []
  test_size = 0.2 #테스트 데이터 크기 여기서는 4개의 테스트 데이터를 뽑음
  start = 0
  end   = df_shu.shape[0] 
  test_ = df_shu.shape[0]*test_size

  for _ in range(int(test_)):
    a = np.random.randint(start,end, size=1)[0]   #랜덤하게 테스트 데이터로 삼을 인덱스 번호를 뽑음
    while a in li:                                #만약 그 번호가 기존에 있는지를 while로 확인
      a = np.random.randint(start,end, size=1)[0] #다시 번호를 뽑고 다시 while으로 돌아간다.
    li.append(a)                                  #번호를 li에 추가한다.
  global X_train
  global X_test
  global y_train
  global y_test

  X_test,y_test   = df_shu.iloc[li,:8],df_shu.iloc[li,8]                                  #테스트 데이터는 그 인덱스 번호로 df_shu에서 슬라이싱해서 만듬 
  X_train,y_train = df_shu.drop(index = li).iloc[:,:8],df_shu.drop(index = li).iloc[:,8]  #훈련 데이터는 그 인덱스 번호를 df_shu에서 제거해서 만든다.






### 미니 배치로 데이터 나누기


In [ ]:
'''
미니 배치로 데이터를 나누기
그냥 전역변수로 미니배치를 담을 mini_1들을 선언해서 
데이터는 이미 섞었으니 그냥 위에서 4개씩 묶는다
'''
def mini_batch():
  #4개씩 x_train을 나누기
  
  global mini_1,mini_2,mini_3,mini_4
  
  mini_1 = X_train.iloc[0:4,:]
  mini_2 = X_train.iloc[4:8,:]
  mini_3 = X_train.iloc[8:12,:]
  mini_4 = X_train.iloc[12:16,:]


### 순전파 과정 함수

In [ ]:
'''
가중치-편향 연산을 위해서 미니 배치마다 가중치-편향 연산을 한다.
가중치 편향 연산된 것에 얼마만큼의 신호로 출력할지를 정하기 위해서 활성화 함수를 취한다. 이진분류이니 sigmoid이다. 
sigmoid는 1 / (1 + exp(-x))
나온 출력값은 tt_loss에 저장한다.
그리고 로지스틱 회귀에서 하는 것처럼 임계값 0.5를 기준으로 출력값이 0.5이상이면 1을 리턴하고 미만이면 0을 리턴하게끔 바꿔서 예측값으로 한다.
예측값을 tt_acc에 저장하고 이를 미니배치 정확도 측정에 쓰기 위해서 predcit_1,2,3,4에 전역변수로 저장한다. 

'''




def forward_propa():
  #각 데이터를 가중치를 곱하고 bias를 더해서 리턴하기
  
  li = [mini_1,mini_2,mini_3,mini_4]
  tt_acc = []
  global tt_loss
  tt_loss = []

  for i in li:
    temp = i * weight                                             # i는 미니배치된 훈련데이터이고 여기에 가중치를 곱해서 가중치 연산을 한다.
    res = temp.apply(np.sum,axis = 1) + bias                      # 가중치 연산된것을 다 더하고 추가로 편향을 더해서 가중치-편향 연산을  한다.
    
    #가중치-편향 연산에 시그모이드 함수를 apply를 쓰기
    res_activated = res.apply(lambda x : 1 / (1 + np.exp(-x)) )
    tt_loss.append(res_activated.tolist())
  
    #조건문으로 0.5이상이면 1을 아니면 0을 리턴하기
    #ans = [1 if i >= 0.5 else 0 for i in res_activated]
    ans = []
    for i in res_activated:
      if i >= 0.5:
        ans.append(1)
      else:
        ans.append(0)
    
    
    tt_acc.append(ans)

  global predict_1,predict_2,predict_3,predict_4

  predict_1 = tt_acc[0]
  predict_2 = tt_acc[1]
  predict_3 = tt_acc[2]
  predict_4 = tt_acc[3]
  


In [ ]:
#forward_propa()
#0인덱스 데이터 가중치-편향 연산 결과 


In [ ]:
#정확도 산출하기

def acc():
  mean_1 = np.mean(y_train[0:4]   == predict_1)
  mean_2 = np.mean(y_train[4:8]   == predict_2)
  mean_3 = np.mean(y_train[8:12]  == predict_3)
  mean_4 = np.mean(y_train[12:16] == predict_4)

  global accuracy

  accuracy = np.mean((mean_1,mean_2,mean_3,mean_4))



In [ ]:
#acc() #모델의 정확도 확인

L
=
−
1
N
∑
N
i
=
1
t
i
l
o
g
(
y
i
)
+
(
1
−
t
i
)
l
o
g
(
1
−
y
i
)
   ...(binary crossentropy)

yi는 예측값, ti는 실제값

In [ ]:

def loss():
  global loss_cross1,loss_cross2,loss_cross3,loss_cross4
  loss_cross1,loss_cross2,loss_cross3,loss_cross4 = 0,0,0,0
  delta = 1e-7
  for i in range(4):
    temp1 = y_train[0:4].tolist()[i] * np.log(tt_loss[0][i]+delta)
    temp2 = y_train[4:8].tolist()[i] * np.log(tt_loss[1][i]+delta)
    temp3 = y_train[8:12].tolist()[i] * np.log(tt_loss[2][i]+delta)
    temp4 = y_train[12:16].tolist()[i] * np.log(tt_loss[3][i]+delta)

    loss_cross1 += temp1
    loss_cross2 += temp2
    loss_cross3 += temp3
    loss_cross4 += temp4


  global loss_cee
  loss_cee = -(loss_cross1 + loss_cross2 + loss_cross3 + loss_cross4) / 4
  #[Epoch 1] TrainData - Loss = 3.500, Accuracy = 0.800
  print(f'TrainData - Loss = {loss_cee}, Accuracy = {accuracy}') 


In [ ]:
def main():
  load_data()
  para()
  data_shu()
  data_split()
  mini_batch()
  forward_propa()
  acc()
  loss()



In [ ]:
main()

TrainData - Loss = 7.703449122378814, Accuracy = 0.8125
